In [114]:
import requests
import re
import string
from bs4 import BeautifulSoup
from dateutil import parser
import csv

In [106]:
r = requests.get('http://www.cic.gc.ca/english/express-entry/past-rounds.asp')
soup = BeautifulSoup(r.text, 'html.parser')

programs_available = ['Federal Skilled Worker Program', 
                      'Federal Skilled Trades Program', 
                      'Canadian Experience Class',
                      'Provincial Nominee Program']


In [122]:
all_size = []
all_points = []
all_dates = []
program_count = []
single_program = []
year_targets = []
for draw_info in soup.find_all("details"):
    # parse draw size and draw points
    td = draw_info.find_all("td")
    all_size.append( int( re.sub("\D", "", td[0].get_text() ) ) )
    all_points.append( int( re.sub("\D", "", td[1].get_text() ) ) )
    
    # parse draw date
    date_dirty = re.sub("[^0-9a-zA-Z \/]+", "", draw_info.find("h3").get_text()).split(" ")
    del date_dirty[0]
    date = parser.parse(" ".join(date_dirty))
    all_dates.append(date.strftime('%d %b %Y'))
    # find which programs were used in this round of invitations and count them
    current_program_count = 0
    try:
        footnote_raw = draw_info.find("dd").get_text()
        current_program_count = 0
        for program in programs_available:
            current_program_count = current_program_count + footnote_raw.lower().count(program.lower())
    except:
        # based on data, we can safely assume that all 4 programs were in use if data is missing
        current_program_count = 4
        pass
    program_count.append(current_program_count)
    if program_count == 1:
        single_program.append(1)
    else:
        single_program.append(0)
    # add year target feature with current knowledge
    if date.year == 2017:
        year_targets.append(300000)
    else:
        year_targets.append(280000)

In [123]:
out = zip(all_dates, all_size, all_points, program_count, single_program, year_targets)

with open('dataset.csv', "wb") as csv_file:
        writer = csv.writer(csv_file, delimiter=',')
        for line in out:
            writer.writerow(line)